In [ ]:
# The goal is to show all the complexities of NN search with small integer numbers
# - global-phase rank-score-drop-limit
# - different result sorts

In [9]:
# Start Vespa with all the required functionality present
!docker run --detach --rm --name vespa-small-numbers --hostname vespa-small-numbers --publish 0.0.0.0:8080:8080 --publish 0.0.0.0:19050:19050 --publish 0.0.0.0:19071:19071 vespaengine/vespa:8.484.1

403bf8ec04dd30a7fc515f944e5e1e2280f9cd5f173f31a89a036ba51ce5f903


In [101]:
# Utility functions for the demo
from typing import MutableMapping
import pandas as pd

def _flatten_dict_gen(d, parent_key, sep):
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            yield from flatten_dict(v, new_key, sep=sep).items()
        else:
            yield new_key, v


def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str = '.'):
    return dict(_flatten_dict_gen(d, parent_key, sep))


def asdf(vespa_response):
    """Convert Vespa response to the DataFrame for presentation"""
    hits = vespa_response.hits
    r_hits = [{
        'position': index,
        'relevance': item['relevance'],
        **flatten_dict(item['fields'])
    } for index, item in enumerate(hits)]
    return pd.DataFrame.from_records(data=r_hits).drop(columns='sddocname')

In [174]:
# Setup a simple Vespa application package
from vespa.package import (ApplicationPackage, Field, Schema, Document, RankProfile, HNSW, GlobalPhaseRanking, Function,
                           QueryProfile, QueryField, QueryProfileType, QueryTypeField)
from vespa.io import VespaResponse

embedding_tensor_type = "tensor<float>(d0[1])"

vap = ApplicationPackage(
    name="annsmallnumbers",
    query_profile_type=QueryProfileType(
        fields=[
            QueryTypeField(
                name="ranking.features.query(q)",
                type= embedding_tensor_type
            )
        ]
    ),
    query_profile=QueryProfile(
        fields=[
            QueryField(name='ranking.matching.approximateThreshold', value='0.99'),
            QueryField(name='hits', value=5),
            QueryField(name='ranking.globalPhase.rankScoreDropLimit', value=0.0),
        ]
    ),
    schema=[
        Schema(
            name="ann",
            document=Document(
                fields=[
                    Field(
                        name="filter",
                        type="int",
                        indexing=["attribute", "summary"],
                        attribute=["fast-search"],
                    ),
                    Field(
                        name="embedding",
                        type=embedding_tensor_type,
                        indexing=["attribute", "index"],
                        ann=HNSW(
                            distance_metric="euclidean",
                            max_links_per_node=16,
                            neighbors_to_explore_at_insert=200,
                        ))
                ]
            ),
            rank_profiles=[
                RankProfile(
                    name="ann",
                    functions=[
                        Function(
                            name='nearest_neighbor_closeness',
                            # rawScore = 1 / (1 + sqrt((q - d)^2))
                            expression='rawScore(embedding)',
                        ),
                    ],
                    first_phase="nearest_neighbor_closeness + 1",
                    match_features=['nearest_neighbor_closeness', 'firstPhase'],
                ),
                RankProfile(
                    name="ann_global_cut",
                    inherits='ann',
                    inputs=[
                        ("query(ann_hits)", "double", "1"),
                    ],
                    first_phase="nearest_neighbor_closeness + 1",
                    global_phase=GlobalPhaseRanking(
                        rerank_count=5,
                        # reciprocal_rank=1.0 / (k + rank), where I set k=0
                        expression="if(reciprocal_rank(nearest_neighbor_closeness, 0) >= 1.0/query(ann_hits) || nearest_neighbor_closeness == 0, firstPhase, 0.0)",
                        # This is not rendered properly in the schema file, but it is covered with the default query profile
                        rank_score_drop_limit=0.0,
                    )
                ),
                RankProfile(
                    name='ann_global_cut_sorted',
                    inherits='ann_global_cut',
                    first_phase="attribute(filter) + 1",
                    global_phase=GlobalPhaseRanking(
                        rerank_count=5,
                        expression="if(reciprocal_rank(nearest_neighbor_closeness, 0) >= 1.0/query(ann_hits) || nearest_neighbor_closeness == 0, firstPhase, 0.0)",
                        rank_score_drop_limit=0.0,
                    )
                ),
                RankProfile(
                    name='ann_global_cut_sorted_asc',
                    inherits='ann_global_cut',
                    first_phase="2147483647 - attribute(filter)", # Some very large number so, that the difference is going to be positive
                    global_phase=GlobalPhaseRanking(
                        rerank_count=5,
                        expression="if(reciprocal_rank(nearest_neighbor_closeness, 0) >= 1.0/query(ann_hits) || nearest_neighbor_closeness == 0, firstPhase, 0.0)",
                        rank_score_drop_limit=0.0,
                    )
                )
            ]
        )
    ]
)

In [175]:
# Deploy VAP zip file
vap_file_name = "global-small-numbers.zip"
vap.to_zipfile(vap_file_name)
! vespa deploy {vap_file_name} -t http://localhost:19071

Uploading application package... done;1m⡿
Success: Deployed 'global-small-numbers.zip' with session ID 22


In [11]:
from vespa.application import Vespa

prod_vespa_host = "http://localhost"
app = Vespa(url=prod_vespa_host, port=8080)

In [32]:
# Create and feed 5 dummy docs
docs = [
    {
        'id': f'{i}',
        'fields': {
            'filter': i,
            'embedding': [i]
        }
    } for i in range(5)]


def callback(response: VespaResponse, document_id: str):
    if not response.is_successful():
        print(f"Error when feeding document {document_id}: {response.get_json()}")


app.feed_iterable(docs, schema="ann", namespace="ann", callback=callback)

In [177]:
# simple query that we would expect fetches 2 hits: 1 lexical and 1 from NN
request_body = {
    'yql': """
        select *
        from ann
        where (
            filter = 0
            OR
            (
                ({targetHits:1}nearestNeighbor(embedding, q))
                AND
                filter >= 3
            )
        )
        """,
    'ranking.profile': 'ann',
    "input.query(q)": [4]
}

In [178]:
# First, let's see what matched when no ranking is involved, so let's try the 'unranked' ranking profile
asdf(app.query(body={
    **request_body,
    'ranking.profile': 'unranked',
}))

,position,relevance,documentid,filter
0,0,0.0,id:ann:ann::0,0
1,1,0.0,id:ann:ann::3,3
2,2,0.0,id:ann:ann::4,4


In [179]:
# but it returns 3 hits! 1 lexical, 2 from NN, yeah and why 5 is not in the results?
# Now what happens when our naive ranking is done
asdf(app.query(body=request_body))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,2.0,2.0,1.0,id:ann:ann::4,4
1,1,1.5,1.5,0.5,id:ann:ann::3,3
2,2,1.0,1.0,0.0,id:ann:ann::0,0


In [168]:
# The same 3 matches, just ranked in a way we want.

In [186]:
# However, when doing ANN we have two hits
asdf(app.query(body={
    **request_body,
    'ranking.matching.approximateThreshold': 0.0,
}))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,2.0,2.0,1.0,id:ann:ann::4,4
1,1,1.0,1.0,0.0,id:ann:ann::0,0


In [187]:
# We're kind of lucky that the best NN came back but that is the nature of such index
# Now let's try with the global phase re-score trick
asdf(app.query(body={
    **request_body,
    'ranking.profile': 'ann_global_cut',
}))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,2.0,2.0,1.0,id:ann:ann::4,4
1,1,1.0,1.0,0.0,id:ann:ann::0,0


In [181]:
# but what happens when you add order by
request_body_order_by = {
    'yql': """
        select *
        from ann
        where (
            filter = 0
            OR
            (
                ({targetHits:1}nearestNeighbor(embedding, q))
                AND
                filter >= 3
            )
        )
        order by filter desc
        """,
    'ranking.profile': 'ann_global_cut',
    "input.query(q)": [4]
}
asdf(app.query(body=request_body_order_by))

VespaError: [{'code': 3, 'summary': 'Illegal query', 'source': 'annsmallnumbers_content', 'message': 'Sorting is not supported with global phase'}]

In [182]:
# Which means that sorting has to be done with ranking
asdf(app.query(body={
    **request_body,
    'ranking.profile': 'ann_global_cut_sorted',
}))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,5.0,5.0,1.0,id:ann:ann::4,4
1,1,1.0,1.0,0.0,id:ann:ann::0,0


In [172]:
# Above we get 1 match from NN one from lexical, and one hit removed  with the `rankScoreDropLimit` while maintaining order on the

In [183]:
# And now let's sort ascending
asdf(app.query(body={
    **request_body,
    'ranking.profile': 'ann_global_cut_sorted_asc',
}))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,2.147484e+09,2.147484e+09,0.0,id:ann:ann::0,0
1,1,2.147484e+09,2.147484e+09,1.0,id:ann:ann::4,4


In [184]:
# And if we don't cut, all the hits are still there
# And now let's sort ascending
asdf(app.query(body={
    **request_body,
    'ranking.profile': 'ann_global_cut_sorted_asc',
    'ranking.globalPhase.rankScoreDropLimit': -10,
}))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,2.147484e+09,2.147484e+09,0.0,id:ann:ann::0,0
1,1,2.147484e+09,2.147484e+09,1.0,id:ann:ann::4,4
2,2,0.000000e+00,2.147484e+09,0.5,id:ann:ann::3,3


In [189]:
# Yeah, this brings back hits in the wrong order, but it is solvable by reranking more NN matches
asdf(app.query(body={
    **request_body,
    'ranking.profile': 'ann_global_cut_sorted_asc',
    'ranking.globalPhase.rankScoreDropLimit': -10,
    "input.query(ann_hits)": 2,
}))

,position,relevance,matchfeatures.firstPhase,matchfeatures.nearest_neighbor_closeness,documentid,filter
0,0,2.147484e+09,2.147484e+09,0.0,id:ann:ann::0,0
1,1,2.147484e+09,2.147484e+09,0.5,id:ann:ann::3,3
2,2,2.147484e+09,2.147484e+09,1.0,id:ann:ann::4,4


In [ ]:
# stop the docker container
! docker stop vespa-small-numbers

In [ ]:
# Params to copy paste
# "input.query(ann_hits)": 1,
# 'ranking.globalPhase.rerankCount': 100,
# 'ranking.globalPhase.rankScoreDropLimit': 1.0,